In [3]:
import tensorflow as tf

<h3 style='color:purple'>Create tf dataset from a list</h3>

In [4]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<h3 style='color:purple'>Iterate through tf dataset</h3>

In [5]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


2025-01-26 17:56:24.070508: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3 style='color:purple'>Iterate through elements as numpy elements</h3>

In [6]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


2025-01-26 17:56:24.706275: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3 style='color:purple'>Iterate through first n elements in tf dataset</h3>

In [7]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<h3 style='color:purple'>Filter sales numbers that are < 0</h3>

In [8]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


2025-01-26 17:56:25.936013: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3 style='color:purple'>Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->72 conversation rate</h3>

In [9]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


<h3 style='color:purple'>Shuffe</h3>

In [10]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2232
2304
2232
2448


https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

<h3 style='color:purple'>Batching</h3>

In [11]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1584 1512]
[2304 2232]
[2232 2448]


<h3 style='color:purple'>Perform all of the above operations in one shot</h3>

In [12]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 2232]
[1512 2304]
[2448 2232]


2025-01-26 17:56:32.723460: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<h3>Images</h3>

In [13]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [14]:
image_count = len(images_ds)
image_count

130

In [15]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [16]:
for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/20 Reasons Why Cats Make the Best Pets....jpg'
b'images/cat/7 Foods Your Cat Can_t Eat.jpg'
b'images/cat/A cat appears to have caught the....jpg'


In [17]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images/dog/20 must-have products for new dog owners.jpg'
b'images/cat/How to buy the best cat food_ according....jpg'
b'images/dog/The 25 Cutest Dog Breeds - Most....jpg'


In [18]:
class_names = ["cat","dog"]

In [19]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [20]:
len(train_ds)

104

In [21]:
len(test_ds)

26

In [49]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-1]

In [50]:
get_label("images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg'>

In [52]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [53]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

2025-01-26 18:04:27.393385: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: images\cat\20 Reasons Why Cats Make the Best Pets....jpg; No such file or directory


NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} images\cat\20 Reasons Why Cats Make the Best Pets....jpg; No such file or directory [Op:ReadFile]

In [ ]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [54]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

TypeError: Cannot iterate over a scalar tensor.

In [55]:
def scale(image, label):
    return image/255, label

In [56]:
train_ds = train_ds.map(scale)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__scale() missing 1 required positional argument: 'label'


In [ ]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())